In [1]:
import geopandas
import pandas as pd
%config InlineBackend.figure_format ='retina'

In [2]:
shpPaths = ['Atlanta - Suburban.shp', 'Atlanta - Urban.shp', 'Austin.shp', 'Boise.shp', 'Charleston.shp', 'Charlotte.shp', 'Colorado Springs.shp', 'Dallas - North.shp', 'Dallas - Suburban.shp', 'Denver.shp', 'Fort Worth.shp', 'Greenville.shp', 'Houston - East.shp', 'Houston - West.shp', 'Jacksonville.shp', 'Las Vegas.shp', 'Nashville.shp', 'North Central Florida.shp', 'NW Arkansas.shp', 'Orlando.shp', 'Phoenix.shp', 'Raleigh - Durham.shp', 'Salt Lake City.shp', 'San Antonio.shp', 'Tampa - St Petersburg - Clearwater.shp', 'West Palm Beach - Boca Raton.shp']

In [2]:
dbfPaths = ['../Shapefiles/Atlanta - Suburban.dbf', '../Shapefiles/Atlanta - Urban.dbf', '../Shapefiles/Austin.dbf', '../Shapefiles/Boise.dbf', '../Shapefiles/Charleston.dbf', '../Shapefiles/Charlotte.dbf', '../Shapefiles/Colorado Springs.dbf', '../Shapefiles/Dallas - North.dbf', '../Shapefiles/Dallas - Suburban.dbf', '../Shapefiles/Denver.dbf', '../Shapefiles/Fort Worth.dbf', '../Shapefiles/Greenville.dbf', '../Shapefiles/Houston - East.dbf', '../Shapefiles/Houston - West.dbf', '../Shapefiles/Jacksonville.dbf', '../Shapefiles/Las Vegas.dbf', '../Shapefiles/Nashville.dbf', '../Shapefiles/North Central Florida.dbf', '../Shapefiles/NW Arkansas.dbf', '../Shapefiles/Orlando.dbf', '../Shapefiles/Phoenix.dbf', '../Shapefiles/Raleigh - Durham.dbf', '../Shapefiles/Salt Lake City.dbf', '../Shapefiles/San Antonio.dbf', '../Shapefiles/Tampa - St Petersburg - Clearwater.dbf', '../Shapefiles/West Palm Beach - Boca Raton.dbf']

In [ ]:
# Reading in each shapefile and adding Area and Boundaries columns
#for path in shpPaths:
#    gdf = geopandas.read_file(path)
#    gdf["Area"] = gdf.area
#    gdf["Boundaries"] = gdf.boundary

In [3]:
import pandas as pd
from simpledbf import Dbf5

# Initialize an empty DataFrame
combined_df = pd.DataFrame()

# Iterate over each DBF file and append it to the combined DataFrame
for dbf_path in dbfPaths:
    dbf = Dbf5(dbf_path)
    df = dbf.to_dataframe()
    
    # Combine 'Submarket' and 'submarket' into a single column
    if 'Submarket' in df.columns and 'submarket' in df.columns:
        df['Submarket'] = df['Submarket'].combine_first(df['submarket'])
        df = df.drop(columns=['submarket'])
    elif 'submarket' in df.columns:
        df.rename(columns={'submarket': 'Submarket'}, inplace=True)
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined DataFrame to a CSV file (optional)
combined_df.to_csv('../combined_dbfs.csv', index=False)

PyTables is not installed. No support for HDF output.


### Cleaning combined DBFs

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../combined_dbfs.csv')

In [5]:
# To lowercase
df['Submarket'] = df['Submarket'].str.lower()

In [6]:
# Remove special characters and standardize
def clean_locality(text):
    text = text.replace('-', '_')
    text = text.replace(' ', '_')
    text = text.replace('/', '_')
    return text

df['Submarket'] = df['Submarket'].apply(clean_locality)

In [11]:
def remove_double(text):
    text = text.replace('__', '_')
    return text

df['Submarket'] = df['Submarket'].apply(remove_double)

In [14]:
df = df[["Submarket", "Market"]]

In [18]:
df.to_csv('../submarkets_list.csv')